In [5]:
import kmeans_qaoa as kq
from qiskit import IBMQ
import numpy as np
import coreset

In [27]:
IBMQ.delete_account()

In [28]:
IBMQ.save_account('c525171861faf3f1a9fc7ce098a5eded9bb68c9864aee62528b157d2ec4123f3d0083516735395460c8d6c80b4c48ab0db549f55bca91dc05d626e2eff21b0c2')

In [29]:
print(IBMQ.stored_account())
print(IBMQ.active_account())

{'token': 'c525171861faf3f1a9fc7ce098a5eded9bb68c9864aee62528b157d2ec4123f3d0083516735395460c8d6c80b4c48ab0db549f55bca91dc05d626e2eff21b0c2', 'url': 'https://auth.quantum-computing.ibm.com/api'}
None


In [30]:
IBMQ.update_account()

Found 1 credentials.
No credentials available for updating could be found. No action will be performed.
Warnings:
The stored account with url "https://auth.quantum-computing.ibm.com/api" is already an IBM Q Experience v2 account.


In [31]:
provider = IBMQ.load_account()

In [6]:
coreset_vectors = np.load('pulsar_coreset_vectors.npy')
coreset_weights = np.load('pulsar_coreset_weights.npy')
for v, w in zip(coreset_vectors, coreset_weights):
    print(v,w)

[ 24.20128166  14.69255179  -0.45390471  -2.50806823 -10.60854682
 -12.77520754   1.85345051  29.7062432 ] 7290.557889102448
[-40.19715584  -5.19405635   1.18948133   2.32577153  -8.87694146
  -1.34954267  -1.06196487 -50.25839323] 5092.263382334067
[-22.97840584 -10.36643817   0.11000542   0.88273192  -8.72894815
  -4.90645825  -1.04824229 -42.5917051 ] 7089.681083956408
[ 15.28721916  -1.58046735  -0.57841962  -1.7482864  -11.93045317
 -17.50827711  13.78608193 485.4010695 ] 216.80510977645045
[ -3.14246834   1.52523931  -0.35035421  -1.81937942 -11.4889816
 -16.47183535   8.83169576 283.2622327 ] 1173.2454254525042


In [ ]:
pulsar_coreset = [(1, np.array([-2,0])), (1, np.array([-1,0])), (1, np.array([1,0])), (1, np.array([2,0]))]
#coreset = None
coreset_points, G, H = kq.gen_coreset_graph(coreset=coreset, metric='dot')
#coreset_points2, G_dist, H2 = kq.gen_coreset_graph(coreset=coreset, metric='dist')
kq.plot_coreset_graph(coreset_points, G)
hstr = ''
for term in H:
    hstr += '{:.1f}{} + '.format(term[0], ''.join(term[1]))
hstr = hstr[:-2]
print('The corresponding problem Hamiltonian is:\nH = {}'.format(hstr))

In [ ]:
data_vectors = np.load('Datasets/pulsar_data_vectors.npy')
print(coreset.get_cost(data_vectors, YOUR_PROPOSED_CLUSTERS))